In [1]:
import glob
import os
import sys
sys.path.append("..")
import pandas as pd
from pandas.io.parsers import read_csv

from utils.configuration import Configuration

config = Configuration()
def add_file_name(csv, path):
    file_name = os.path.basename(path).split('.')[0]
    csv['config'] = file_name.split('result_')[1]
    return csv

In [2]:
def read_results():
    desc_map_files = []
    for path in glob.glob('../'+config.results_dir + "/*.csv"):
        csv = read_csv(path, encoding='latin-1')
        csv = add_file_name(csv, path)
        desc_map_files.append(csv)
    results= pd.concat(desc_map_files, axis=0, ignore_index=True)
    results.drop('target_app', inplace=True, axis=1)
    results.drop('src_app', inplace=True, axis=1)
    results.fillna(0, inplace=True)
    return results

In [3]:
def config_average_performance(results):
    return results.groupby('config').mean().sort_values(by = ['accuracy'], ascending=False)

In [4]:
results = read_results()
agg_results = config_average_performance(results).sort_values(by = ['f1_score'], ascending=False)

In [5]:
# config_stat = read_csv('../'+config.analyse_dir + '/configs_status.csv')
# config_stat.set_index('config', inplace=True)
# final_result = config_stat.join(agg_results)
final_result = agg_results
final_result

,tp,tn,fp,fn,effort,accuracy,precision,recall,f1_score,reduction
config,,,,,,,,,,
wm_googleplay_craftdroid_intersection,2.419355,0.548387,1.354839,1.193548,2.048387,0.632291,0.654263,0.689516,0.662062,0.557258
jaccard_empty_craftdroid_atm,2.435484,0.548387,1.483871,1.177419,2.064516,0.628281,0.651843,0.686559,0.659414,0.543280
jaccard_empty_custom_intersection,2.387097,0.548387,1.483871,1.225806,2.048387,0.630554,0.651997,0.682527,0.657953,0.550000
wm_googleplay_craftdroid_atm,2.403226,0.548387,1.451613,1.209677,2.112903,0.628320,0.648349,0.685484,0.656942,0.539785
jaccard_empty_craftdroid_intersection,2.419355,0.548387,1.532258,1.193548,2.080645,0.621830,0.646467,0.681183,0.654038,0.537903
...,...,...,...,...,...,...,...,...,...,...
glove_standard_custom_union,1.532258,0.548387,2.032258,2.080645,2.983871,0.401211,0.420020,0.444086,0.420543,0.305197
w2v_blogs_custom_craftdroid,1.508197,0.557377,1.967213,2.114754,2.950820,0.398584,0.414338,0.433060,0.412981,0.309654
glove_blogs_custom_craftdroid,1.540984,0.557377,2.098361,2.081967,3.000000,0.389036,0.408873,0.435519,0.410745,0.293807


In [6]:
configs_mrr = read_csv('../' + config.analyse_dir + '/mrr.csv')
configs_mrr['config'] = configs_mrr[['word_embedding', 'training_set', 'algorithm','descriptors']].agg('_'.join, axis=1)
configs_mrr.drop(columns=['descriptors','word_embedding', 'training_set', 'algorithm'], inplace=True)
configs_mrr.set_index('config', inplace=True)
final_result = final_result.join(configs_mrr).sort_values(by = ['f1_score'], ascending=False)
# configs_mrr.loc[configs_mrr['config'] == 'glove_android_atm_union']
final_result.to_csv('../' + config.analyse_dir + '/final_results.csv')